In [18]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import cv2,math,gc

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.nn import Parameter
import torch.optim as optim

!pip install "../input/efficient-net/dist/efficientnet_pytorch-0.7.0.tar"
from efficientnet_pytorch import EfficientNet

!pip install "../input/faissgpuwheel/faiss_gpu-1.7.0-cp37-cp37m-manylinux2014_x86_64.whl"
import faiss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

import warnings
warnings.simplefilter('ignore')
    
torch.backends.cudnn.benchmark = True
from transformers import (BertTokenizer, BertModel,
                          DistilBertTokenizer, DistilBertModel)
from sklearn.preprocessing import LabelEncoder
from tqdm.autonotebook import tqdm

Processing /kaggle/input/efficient-net/dist/efficientnet_pytorch-0.7.0.tar
DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.1 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16033 sha256=ca59db0a5d4da10e47b08a082320dbaa37994b6db10151a59da76160b0febc62
  Stored in directory: /root/.cache/pip/wheels/af/8c/80/1bf8cc2fa471c320978f34c5290675daaa96446e1b9ba45555
Successfully built efficientnet-pytorch
  Attempting uninstall: efficientnet-pytorch
    Found existing installation: efficientnet-pytorch 0.7.0
    Uninstalling efficientnet-pytorch-0.7.0:
      Successfully uninstalled efficientnet-pytorch-0.7.0
Processing /kaggle/input/faissgpuwheel/faiss_gpu-1.7.0-cp37-cp37m

In [19]:
TRAIN = True

In [20]:
df = pd.read_csv("../input/shopee-product-matching/train.csv")
labels = list(set(df.label_group.values))
labels.sort()
labels_length = len(labels)
single_fold_length = labels_length//5 + 1
rank = dict()
for i, label in enumerate(labels):
    rank[label] = i
df['rank'] = df.label_group.map(rank)
df['fold'] = df['rank'].apply(lambda x: x//single_fold_length)
df.head()

,posting_id,image,image_phash,title,label_group,rank,fold
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,666,0
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,7572,3
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,6172,2
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,10509,4
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,9425,4


In [21]:
class cfg:
    img_size = (380,380)
    feavec_num1 = 512
    feavec_num2 = 768
    fea_norm = 64
    margin = 0.35
    batch = 16
    mname = 'efficientnet-b3'
    clsize = 8812
    lr = 0.001
    momentum = 0.9
    weight_decay = 0.0005
    log_interval = 1000
    epochs = 10
    wpath = '../input/my-weight/efficientnet-b3_eff_bert_arcface_epoch_10.pt'

In [22]:
class CFG:
    DistilBERT = True # if set to False, BERT model will be used
    bert_hidden_size = 768
    
    batch_size = 64
    epochs = 100
    num_workers = 4
    learning_rate = 1e-5 
    scheduler = "ReduceLROnPlateau"
    step = 'epoch'
    patience = 2
    factor = 0.8
    dropout = 0.5
    model_path = "/kaggle/working"
    max_length = 30
    model_save_name = "bert_model.pt"
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [23]:
tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
df['target'] = df.label_group.map(tmp)
df['target'] = df['target'].apply(lambda x: ' '.join(x))
df_cu = cudf.DataFrame(df)

print('df shape is', df.shape )
df.head()

df shape is (34250, 8)


,posting_id,image,image_phash,title,label_group,rank,fold,target
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,666,0,train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,7572,3,train_3386243561 train_3423213080
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,6172,2,train_2288590299 train_3803689425
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,10509,4,train_2406599165 train_3342059966
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,9425,4,train_3369186413 train_921438619


In [24]:
df1 = df[df.fold!=4]
df1_val = df[df.fold==4]
df1_val['label'] = 1
ranks = list(set(df1["rank"].values))
ranks.sort()
ranks_length = len(ranks)
print(ranks_length)
label = dict()
for i, rank in enumerate(ranks):
    label[rank] = i
df1['label'] = df1["rank"].map(label)
df1.head()

8812


,posting_id,image,image_phash,title,label_group,rank,fold,target,label
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,666,0,train_129225211 train_2278313361,666
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,7572,3,train_3386243561 train_3423213080,7572
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,6172,2,train_2288590299 train_3803689425,6172
5,train_2464356923,0013e7355ffc5ff8fb1ccad3e42d92fe.jpg,bbd097a7870f4a50,CELANA WANITA (BB 45-84 KG)Harem wanita (bisa...,2660605217,6836,3,train_2464356923 train_2753295474 train_305884580,6836
6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137,4687,2,train_1802986387 train_1396161074 train_713073...,4687


In [25]:
if CFG.DistilBERT:
    model_name='cahya/distilbert-base-indonesian'
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    bert_model = DistilBertModel.from_pretrained(model_name)
else:
    model_name='cahya/bert-base-indonesian-522M'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name)

In [26]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.30, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output  #need softmax then


class Model(nn.Module):
    def __init__(self,name,clustersize,feavec, bert_model, last_hidden_size=CFG.bert_hidden_size):
        super(Model, self).__init__()
        self.eff = EfficientNet.from_pretrained(name)
        self.bert_model = bert_model
        self.out = nn.Linear(1000+last_hidden_size,feavec)
        self.margin = ArcMarginProduct(in_features=feavec, 
                                       out_features = clustersize, 
                                       s=cfg.fea_norm, 
                                       m=cfg.margin)      
    
    def get_bert_features(self, batch):
        output = self.bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        last_hidden_state = output.last_hidden_state # shape: (batch_size, seq_length, bert_hidden_dim)
        CLS_token_state = last_hidden_state[:, 0, :] # obtaining CLS token state which is the first token.
        return CLS_token_state

    def forward(self, batch, labels=None):
        x1 = self.eff(batch['image'])
        x2 = self.get_bert_features(batch)
        
        x = torch.hstack((x1, x2))
        x = self.out(x)
        if labels is not None:
            return self.margin(x,labels)
        return F.normalize(x,dim=1)

In [27]:
model = Model(name=cfg.mname,clustersize=cfg.clsize, feavec=512, bert_model=bert_model).to(device)
model.load_state_dict(torch.load(cfg.wpath, map_location=device))
optimizer = optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.momentum, weight_decay=cfg.weight_decay)

Loaded pretrained weights for efficientnet-b3


In [28]:
def load_image(file_name):
    if TRAIN:
        file_path = f'/kaggle/input/shopee-product-matching/train_images/{file_name}'
    else:
        file_path = f'/kaggle/input/shopee-product-matching/test_images/{file_name}'
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, cfg.img_size)
    tensor_img = torch.tensor(img)
    tensor_img = tensor_img.permute(( 2, 0, 1)).float()/255.0
    return tensor_img

In [29]:
class MyDataset(Dataset):
    def __init__(self, dataframe, tokenizer, mode="train", max_length=None):
        self.dataframe = dataframe
        if mode != "test":
            self.targets = dataframe['label'].values
        texts = list(dataframe['title'].apply(lambda o: str(o)).values)
        self.encodings = tokenizer(texts, 
                                   padding=True, 
                                   truncation=True, 
                                   max_length=max_length)
        self.mode = mode
        self.img = dataframe.image.values
        
    def __getitem__(self, idx):
        # putting each tensor in front of the corresponding key from the tokenizer
        # HuggingFace tokenizers give you whatever you need to feed to the corresponding model
        item = {key: torch.tensor(values[idx]) for key, values in self.encodings.items()}
        # when testing, there are no targets so we won't do the following
        if self.mode != "test":
            item['labels'] = torch.tensor(self.targets[idx]).long()
        img = self.img[idx]
        img = load_image(img)
        item['image'] = img
        return item
    
    def __len__(self):
        return len(self.dataframe)

In [30]:
train_dataset = MyDataset(df1, tokenizer, mode='train', max_length=CFG.max_length)
train_loader = DataLoader(train_dataset,
                    batch_size=cfg.batch,
                    shuffle=False,
                    num_workers=2,
                    pin_memory=True,
                    drop_last=False)
val_dataset = MyDataset(df1_val, tokenizer, mode='train', max_length=CFG.max_length)
val_loader = DataLoader(val_dataset,
                    batch_size=cfg.batch,
                    shuffle=False,
                    num_workers=2,
                    pin_memory=True,
                    drop_last=False)

In [31]:
def train(epoch):
    model.train()
    epoch_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        batch = {k: v.to(CFG.device) for k, v in batch.items()}
        images = batch['image']
        label = batch['labels']
        optimizer.zero_grad()
        output = model(batch, label)
        loss = nn.CrossEntropyLoss()(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % cfg.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        epoch_loss = epoch_loss + loss.item()
    return epoch_loss/len(train_loader)

In [32]:
def all_embeddings(df):
    dataset = MyDataset(df, tokenizer, mode='train', max_length=CFG.max_length)
    loader = DataLoader(dataset,
                        batch_size=cfg.batch,
                        shuffle=False,
                        num_workers=2,
                        pin_memory=True,
                        drop_last=False)
    
    model.eval()
    print('start collection')
    feavec = 512
    embedded1 = np.empty((0,feavec),dtype='float32')
    with torch.no_grad():
        for idx,batch in enumerate(loader):
            batch = {k: v.to(CFG.device) for k, v in batch.items()}
            outputs = model(batch)
            embedded1 = np.append(embedded1, outputs.cpu().detach().numpy(),axis=0)

            if idx%100==0:
                print(idx,len(loader))
                print(embedded1.shape)
    return embedded1

In [33]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

def predict_img(df,embeddings,topk=50,threshold=0.63):
    N,D = embeddings.shape
    cpu_index = faiss.IndexFlatL2(D)
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)
    gpu_index.add(embeddings)
    cluster_distance,cluster_index = gpu_index.search(x=embeddings, k=topk)
    
    df['pred_images'] = ''
    pred = []
    for k in range(embeddings.shape[0]):
        idx = np.where(cluster_distance[k,] < threshold)[0]
        ids = cluster_index[k,idx]
        #posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        posting_ids = df['posting_id'].iloc[ids].values
        pred.append(posting_ids)
    df['pred_images'] = pred
    
    df['pred_imgonly'] = df.pred_images.apply(lambda x: ' '.join(x))
    df['f1_img'] = f1_score(df['target'], df['pred_imgonly'])
    score = df['f1_img'].mean()
    #print(f'Our f1 score for threshold {threshold} is {score}')
    return score

In [ ]:
loss_list = []
train_f1_list = []
val_f1_list = []
for epoch in range(1,cfg.epochs + 1):
    loss_list.append(train(epoch))
    
    embeddings_train = all_embeddings(df1)
    embeddings_val = all_embeddings(df1_val)
    train_f1_score = predict_img(df1,embeddings_train, topk=50,threshold=0.88)
    val_f1_score = predict_img(df1_val,embeddings_val, topk=50,threshold=0.88)
    
    train_f1_list.append(train_f1_score)
    val_f1_list.append(val_f1_score)
    
    torch.save(model.state_dict(), "{}_eff_bert_arcface_epoch_{}.pt".format(cfg.mname, epoch))
    print("train_f1_score:", train_f1_score)
    print("val_f1_score:", val_f1_score)

loss_list = np.array(loss_list)
train_f1_list = np.array(train_f1_list)
val_f1_list = np.array(val_f1_list)

np.save('image_only_train_loss.npy', loss_list)
np.save('image_only_train_f1.npy', train_f1_list)
np.save('image_only_val_f1.npy', val_f1_list)

In [ ]:
plt.plot(loss_list)

In [ ]:
plt.plot(train_f1_list)

In [ ]:
plt.plot(val_f1_list)

In [ ]:
for thre in np.arange(70, 120)*0.01:
    val_f1_score = predict_img(df1_val,embeddings_val, topk=50,threshold=thre)
    print(thre, ':', val_f1_score)

In [ ]:
np.arange(70, 100)*0.01

In [35]:
embeddings_val = all_embeddings(df1_val)

start collection
0 425
(16, 512)
100 425
(1616, 512)
200 425
(3216, 512)
300 425
(4816, 512)
400 425
(6416, 512)


In [42]:
val_f1_score = predict_img(df1_val,embeddings_val, topk=50,threshold=0.6)
print( ':', val_f1_score)

: 0.7156906746387467
